<h1 style="font-size: 36px; color: #FFD700">2 - A Gentle Introduction to torch.autograd</h1>

BACKGROUND

Neural network - là tập hợp các hàm lồng nhau đc thực thi trên một số dữ liệu đầu vào. Các hàm này đc xác định bởi các tham số (weight & bias).

Forward Propagation: Trong quá trình này, NN sẽ đưa ra dự đoán tốt nhất về đầu ra dựa vào những gì nó đã học được cho đến thời điểm đó.

=> nhìn đầu vào -> dự đoán đầu ra trên trọng số hiện tại 

=> cố gắng dự đoán đúng nhất có thể với cái mà nó chưa biết đúng sai (theo niềm tin của nó)

Backward Propagation: NN điều chỉnh các tham số của nó theo tỷ lệ với lỗi trong dự đoán của nó. Nó thực hiện bằng cách duyệt ngược từ đầu ra, thu thập các đạo hàm của lỗi liên quan đến các tham số của các hàm gradient và tối ưu jpas bằng cách dùng gradient descent.

Ví dụ:
- Dùng mô hình ResNet18 đã được huấn luyện sẵn từ thư viện torchvision
- Tạo dữ liệu đầu vào ngẫu nhiên: 1 ảnh có kích thước 3×64×64 (RGB, cao & rộng là 64)
- Gán một label ngẫu nhiên (để giả lập nhãn ảnh)

Vì ResNet18 được huấn luyện trên ImageNet, nên đầu ra của nó có dạng (1, 1000) — tức là dự đoán cho 1000 lớp.

In [2]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

In [3]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)

data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Bước Forward Propagation - Ta chạy input data vào mô hình qua từng lớp để đưa ra dự đoán

In [4]:
prediction = model(data) # forward pass

Ta dùng pred + label tương ứng đế tính loss. Tiếp theo là backward qua mạng. 

Truyền ngược bắt đầu khi ta gọi tensor loss.backward() sau đó Autograd tính toán và lưu trữ các grad cho từng tham số cho mô hình trong thuộc tính .grad của tham số.

In [5]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Ta sẽ tải một optimizer (SGD) với lr = 0.01, momentum=0.9 ta truyền tất cả các tham số của mô hình trong trình tối ưu hóa

In [6]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Sau đó ta gọi .step() để bắt đầu gradient descent. Optimizer điều chỉnh từng tham số theo grad được lưu trữ trong .grad

In [7]:
optim.step() #gradient descent

AUTOGRAD

Ta tạo ra 2 tensor a và b với requires_grad=True (mọi hđ trên autograd đều phải theo dõi)

In [8]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4,], requires_grad=True)

Ta tạo ra một tensor Q từ a và b

In [9]:
Q = 3 * a**3 - b**2

Giả sử a và b là các tham số của NN và Q là loss. Trong qtrinh train, ta muốn có các grad của loss theo các tham số 

$\frac{\partial Q}{\partial a} = 9a^2$

$\frac{\partial Q}{\partial b} = -2b$ 

Khi ta gọi Q.backward, autograd sẽ tính toán các grad và lưu trữ trong trong thuộc tính grad tensor tương ứng. 

Ta cần truyền một gradient đối số một cách rõ ràng Q.backward() vì nó là một vector. gradient là một tensor có cùng hình dạng với Q và nó biểu diễn độ dốc của Q so với chính nó, tức là $\frac{dQ}{dQ} = 1$

- Nếu Q là scalar thì không cần truyền gì và ngược alij thì truyền thêm gradient vào.
- Tensor gradient truyền vào có cùng shape với Q
- Mục đích truyền vào để xác định grad theo phần tử nào

Tương đương, chúng ta cũng có thể tổng hợp Q thành một số vô hướng và gọi ngược lại một cách ngầm định, như Q.sum().backward()

In [10]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Các gradient hiện được gửi vào a.grad và b.grad

In [11]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


VECTOR CALCULUS USING AUTOGRAD

Nếu ta có một hàm vector $\vec{y} = f(\vec{x})$ thì gradient của $\vec{y}$ theo $vec{x}$ là ma trận Jacobian J

$$
J = \left( \frac{\partial \vec{y}}{\partial x_1} \ \cdots \ \frac{\partial \vec{y}}{\partial x_n} \right)
=
\begin{pmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
$$

torch.autograd là công cụ để tính tích vector Jacobian - nghĩa là bất cứ vector $\vec{v}$ nào tính đều tính tích $J^T \cdot \vec{v}$


Nếu $\vec{v}$ tình cờ là gradient của một hàm vô hướng $l = g(\vec{y}):$ thì

$$
\vec{v} = \left( \frac{\partial l}{\partial y_1} \ \cdots \ \frac{\partial l}{\partial y_m} \right)^T
$$


- Hàm vô hướng: chỉ nhận một giá trị đầu ra (scala). Vd hàm loss nhận một vector $\vec{y}$ và trả về một sô thực l.
- Gradient của hàm vô hướng đó theo $\vec{y}$:
    - Gradient ở đây chính là vector đạo hàm riêng của l theo từng phần của $\vec{y}$ ký hiệu: $\vec{v} = \nabla_{\vec{y}} \, l$
    - Nghĩa là: 
    $$
    \vec{v} = \left[ \frac{\partial l}{\partial y_1}, \ \frac{\partial l}{\partial y_2}, \ \cdots, \ \frac{\partial l}{\partial y_m} \right]
    $$
- Ý nghĩa trong chain rule:
    - Nếu $\vec{y} = f(\vec{x})$, tức $\vec{y}$ phụ thuộc vào $\vec{x}$, thì gradient của l theo $\vec{x}$ là: $\nabla_{\vec{x}} \, l = J^T \cdot \vec{v}$
    - Trong đó J là ma trận  Jacobian của $\vec{y}$ theo $\vec{x}$


(Chain rule) -Tích vector Jacobian là gradient của l theo $\vec{x} là:
$$
J^T \cdot \vec{v} =
\begin{pmatrix}
\frac{\partial \text{v\_đi}_1}{\partial x_1} & \cdots & \frac{\partial \text{v\_đi}_i}{\partial x_1} \\
\vdots & \ddots & \vdots \\
\frac{\partial \text{v\_đi}_1}{\partial x_N} & \cdots & \frac{\partial \text{v\_đi}_i}{\partial x_N}
\end{pmatrix}
\begin{pmatrix}
\frac{\partial \text{một}}{\partial \text{v\_đi}_1} \\
\vdots \\
\frac{\partial \text{một}}{\partial \text{v\_đi}_i}
\end{pmatrix}
=
\begin{pmatrix}
\frac{\partial \text{một}}{\partial x_1} \\
\vdots \\
\frac{\partial \text{một}}{\partial x_N}
\end{pmatrix}
$$

COMPUTATIONAL GRAPH

autograd lưu trữ một bản ghi dữ liệu (tensor) và tất cả các hoạt động được thực hiện cùng với các tensor kết quả trong một đồ thị có hướng không có chu trình (DAG) bao gồm các function obj. 

Trong DAG:
- lá là các input tensor, gốc là output tensor. 
- theo dõi từ gốc đến lá => có thể tự động tính toán gradient bằng các sử dụng chain rule

autograd thực hiện hai việc cùng lúc trong mỗi lần chuyển tiếp:
- chạy hoạt động được yêu cầu tính toán một tensor kết quả
- duy trì hàm grad của hoạt động đó trog DAG

Backward propagation bắt đầu khi gọi .backward() ở gốc DAG. Sau đó:
- Tính các gradient từ mỗi .grad_fn
- tích lũy các giá trị gradient trong thuộc tính tensor .grad
- sử dụng quy tắc chuỗi, lan truyền đến các leaf-tensor 

Biểu diễn trực quan của DAG:

<img src="https://pytorch.org/tutorials/_images/dag_autograd.png" alt="Ảnh online" style="background-color:white; padding:10px;">


Giải thích:
- Hai ô màu xanh dương là input tensor
- Theo hương mũi tên là Forward 
- Các ô màu xám là các phép toán
- Ô màu xanh là phép trừ với đầu vào là từ hai nhánh

Ví dụ:
Ở ô xanh dương đầu tiền có 2 biến đầu vào là x a

Ở ô màu xanh dương thứ 2 có biến đầu vào là y

Cột trái:
- PowBackward(): $x^2$
- MulBackward(): a * $x^2$

Cột phải:
- PowBackward(): $y^2$

SubBackward(): a * $x^2$ - $y^2$

=> Đây là các Pytorch xây dựng đồ thì để tính toán grad cho z với từng input

Khi gọi backward() - z.backward()
- Bắt đầu từ SubBackward() - cuối đồ thị 
- Quay lai từng node và tự áp dụng quy tắt đạo hàm tương tự.
- Cập nhật .grad cho từng tensor input

Note: Việc lưu trữ grad là để cập nhật trong bước optimizer.step(). Còn bước forward hay backward là để tính toán

Exclusion from the DAG

torch.autograd sẽ theo dõi các hoạt động tất cả tensor có requires_grad mà cờ = True. Ngược lại (False) sẽ loại tensor đó ra khỏi DAG tính gradient 

output tensỏ của một phép toán vẫn yêu cầu gradient khi chỉ có một input set requires_grad=True

In [12]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients?: False
Does `b` require gradients?: True


- Nếu ta set requires_grad = False cho tất cả input:
    - Mô hình đang ở chế độ prediction
    - Nếu ở chế độ train thì không học đc gì hết

Các tham số không tính đạo hàm được gọi là "frozen parameters" - hữu ích khi ta biết những tham số không cần đạo hàm 

=> mang lại lợi ích về hiệu suất khi giảm các phép tính tự động

Trong tinh chỉnh models, ta thường sẽ đóng băng hầu hết mô hình, thường chỉ sửa đổi lớp phân loại để đưa ra dự đoán về nhãn mới. 

In [13]:
# Ta tải model resnet18 đã đc train và đóng băng tất cả các tham số
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Vd: ta tinh chỉnh mô hình trên một tập huấn luyện vs 10 nhãn. 

Trong resnet, bộ phân loại là lớp tuyến tính cuối cùng (model.fc) Chúng ta có thể chỉ cần thay thế nó bằng một lớp tuyến tính mới (mặc định là không đóng băng) đóng vai trò là bộ phân loại của chúng ta

In [14]:
model.fc = nn.Linear(512, 10)

Bây giờ các tham số trong mô hình, ngoại trừ các tham số của model.fc, đều bị đóng băng. Các tham số duy nhất tính gradient là weights và bias của model.fc

In [15]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Lưu ý: 

Dù ta đăng ký tất cả các tham số trong optimizer, nhưng các tham số duy nhất tính gradient (và do đó được cập nhật trong quá trình giảm gradients) là weights và bias của trình phân loại.